In [52]:

DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"
DB_NAME = "loadbug"

In [53]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from neo4j import GraphDatabase # Python database driver
from graphdatascience import GraphDataScience # Python GDS client

# And some cermony to create the driver and gds objects
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS))
gds = GraphDataScience(DB_ULR, auth=(DB_USER, DB_PASS))
gds.set_database(DB_NAME)

# Create databse and schema

In [54]:
# Create (or replace) database
with driver.session(database = "system") as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            "CREATE OR REPLACE DATABASE {dbname}".format(dbname = DB_NAME)
        ).data()
    )

In [55]:
gds.run_cypher("""
    CREATE CONSTRAINT FOR (n:Person) REQUIRE (n.id) IS NODE KEY 
    """)

""


In [56]:
gds.run_cypher("""
    CREATE CONSTRAINT FOR (n:Item) REQUIRE (n.id) IS NODE KEY 
    """)

""


In [57]:
gds.run_cypher("""
    UNWIND RANGE(1, 500000) as id
    CREATE (a:Item{id:id}) 
    """)

""


In [58]:
gds.run_cypher("""
    UNWIND RANGE(1, 1000) as id
    CREATE (a:Person{id:id}) 
    """)

""


In [59]:
gds.run_cypher("""
    UNWIND RANGE(1, 500000) as id
    WITH id%1000 as x
    MATCH (i:Item{id:x})
    MATCH (p:Person{id:x})
    CREATE (p)-[:VIEWED]->(i) 
    """)

""


In [61]:
gds.run_cypher("""
    UNWIND RANGE(1, 500000) as id
    WITH id%1000 as x,  (id%1000)*2 as y
    MATCH (i:Item{id:y})
    MATCH (p:Person{id:x})
    CREATE (p)-[:VIEWED]->(i) 
    """)

""


In [63]:
G, project_stats = gds.graph.project.cypher(
    'items',
    """
    MATCH (i:Item) return id(i) as id
    """,
    """
        MATCH (i1:Item)<-[:VIEWED]-(p:Person)-[:VIEWED]->(i2:Item)
        WHERE id(i1) < id(i2)
        RETURN id(i1) as target, id(i2) as source, 1.0 as weight
     """,
    readConcurrency=16
)
project_stats

Loading:   0%|          | 0/100 [00:00<?, ?%/s]

ClientError: Failed to invoke procedure `gds.beta.listProgress`: Caused by: java.lang.IllegalArgumentException: count is negative: -5

In [64]:
G = gds.graph.get('items')
G.drop()

graphName                                                        items
database                                                       loadbug
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                       500000
relationshipCount                                            249750000
configuration        {'relationshipQuery': 'MATCH (i1:Item)<-[:VIEW...
density                                                       0.000999
creationTime                       2022-08-02T10:24:13.282899000+02:00
modificationTime                   2022-08-02T10:26:03.028976000+02:00
schema               {'graphProperties': {}, 'relationships': {'__A...
Name: 0, dtype: object